In [1]:
#importing all libraries
import pyspark
from pyspark.sql.functions import when
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import when
from pyspark.sql import SparkSession
from datetime import datetime
from pyspark.sql.functions import col
from pyspark.sql.types import TimestampType
from pyspark.sql.functions import to_timestamp
import os, glob
from datetime import date

In [2]:
#getting the spark session instance 
spark=SparkSession.builder.appName("nyc").getOrCreate()

In [3]:
# getting current date, month and day
today = date.today()
year = (today.strftime("%Y"))
month = (today.strftime("%B"))[:3]
day = (today.strftime("%d"))
print(year, month, day)

2022 Apr 28


In [4]:
# year = 2021
# month =
# day = 

In [5]:
############### This is main PATH for nyc text data #####################
FETCH_FOLDER = "/opt/notebooks/nyc/raw_zone"

In [6]:
############### This cell is for joining the PATH with RAW_ZONE #####################
def trip_path_join(new_path):
    return os.path.join(FETCH_FOLDER+new_path)

In [7]:
############### Reading and Converting into DataFrame #####################
def convert_parquet_to_dataframe(trips_complete_path):
    try:
        df=spark.read.parquet(trips_complete_path)
    except FileNotFoundError as e:
        df = None
        print("no such file")
    return df

In [8]:
##########        Testing      #####################



print(glob.glob(trip_path_join("/master/ratecodes/*.parquet")))
# trips_parquet_path = os.path.join(trip_path_join("/trips/2021/Nov/26/*.parquet"))
# df = convert_parquet_to_dataframe(trips_parquet_path)
# print(df.show(5))

['/opt/notebooks/nyc/raw_zone/master/ratecodes/2022_04_28_1651148941115_0.parquet', '/opt/notebooks/nyc/raw_zone/master/ratecodes/2022_04_28_1651148981265_0.parquet']


# Working With  Path

### working for path ( ratecode )

In [9]:
ratecode_val = False
if (trip_path_join("/master/ratecodes/")):
    if len(glob.glob(trip_path_join("/master/ratecodes/*"))):
        ratecode_val = True
        ratecode_df = convert_parquet_to_dataframe(trip_path_join("/master/ratecodes/*.parquet"))

        
print(ratecode_val)

True


### working for path ( payments )

In [10]:
payment_val = False
if (trip_path_join("/master/payments/")):
    if len(glob.glob(trip_path_join("/master/payments/*"))):
        payment_val = True
        payment_df = convert_parquet_to_dataframe(trip_path_join("/master/payments/*.parquet"))

        
print(payment_val)

True


### working for path ( coordinates )

In [11]:
coordinate_val = False
if (trip_path_join("/master/coordinates/")):
    if len(glob.glob(trip_path_join("/master/coordinates/*"))):
        coordinate_val = True
        coordinate_df = convert_parquet_to_dataframe(trip_path_join("/master/coordinates/*.parquet"))

        
print(coordinate_val)

True


### working for path ( zone )

In [12]:
zone_val = False
if (trip_path_join("/master/zone/")):
    if len(glob.glob(trip_path_join("/master/zone/*"))):
        zone_val = True
        zone_df = convert_parquet_to_dataframe(trip_path_join("/master/zone/*.parquet"))

        
print(zone_val)

True


In [13]:
# glob.glob(trip_path_join("/trips/{0}/{1}/{2}/*".format(2021,'Nov',29)))
# print(trip_path_join("/trips/{0}/{1}/{2}/*".format(2021,'Nov',29)))

### working for path ( Trips )

In [14]:
trip_val = False
if (os.path.exists(trip_path_join("/trips/{0}".format(year)))):
    print(year)
    if (os.path.exists(trip_path_join("/trips/{0}/{1}".format(year,month)))):
        print(month)
        if (trip_path_join("/trips/{0}/{1}/{2}".format(year,month,day))):
            if len(glob.glob(trip_path_join("/trips/{0}/{1}/{2}/*".format(year,month,day)))):
                trip_val = True
                
                
                #################################################################
                #            Reading Trip Parquet DataSet on Current Date
                #################################################################
                yellow_taxi_trip_df = convert_parquet_to_dataframe(os.path.join(trip_path_join("/trips/{0}/{1}/{2}/*.parquet".format(year,month,day))))
                
            
                
                #################################################################
                #            
                #################################################################
                # this dataframe contains necessary information related to pick up joined with trip_data
                pickup_zone = yellow_taxi_trip_df.join(zone_df, yellow_taxi_trip_df.pu_location_id == zone_df.location_id,
                                       "inner").select("trip_id", "vendor_id",
                                                       "tpep_pickup_datetime", "passenger_count", "trip_distance",
                                                       "rate_code_id", col("borough").alias("pickup_borough"),
                                                       col("zone").alias("pickup_zone"),
                                                       col("service_zone").alias("pickup_service_zone"),
                                                       "store_and_fwd_flag", "pu_location_id")

                #################################################################
                #            
                #################################################################
                # this dataframe contains the drop related data joined with trip data 
                drop_zone_df = yellow_taxi_trip_df.join(
                    zone_df, yellow_taxi_trip_df.do_location_id == zone_df.location_id,
                    "inner").select("trip_id", "congestion_surcharge", "total_amount",
                                    "improvement_surcharge", "tolls_amount",
                                    "tip_amount", "mta_tax", "extra", "fare_amount",
                                    "payment_type_id", "do_location_id", "tpep_dropoff_datetime",
                                    col("zone").alias("drop_zone"),
                                    col("borough").alias("drop_borough"),
                                    col("service_zone").alias("drop_service_zone"))
                #################################################################
                #            
                #################################################################
                # this dataframe containes the combine information of pick up and delivery
                pickup_drop_df = pickup_zone.join(drop_zone_df, ["trip_id"])
                
                
                #################################################################
                #            
                #################################################################
                ratecode_without_dates=ratecode_df.drop("created_at","updated_at")
                
                
                
                #################################################################
                #            
                #################################################################
                # dropping created_at and updated_at columns from payment dataframe to avoid column redundancy in the final dataframe
                payment_type_without_dates=payment_df.drop("created_at","updated_at")
                
                
                
                
                #################################################################
                #            
                #################################################################
                # this dataframe contains combine information of rates,payment type 
                combined_df = pickup_drop_df.join(ratecode_without_dates, ["rate_code_id"]).join(payment_type_without_dates, ["payment_type_id"])
                
                #################################################################
                #            
                #################################################################
                # this dataframe contains pickup cordinates
                cordinates_pickup = coordinate_df.select(col("coordinate_id").alias("pickup_coordinate_id"),
                      col("latitude").alias("pickup_lattitude"),
                      col("longitude").alias("pickup_longitude"),
                      col("country_name").alias("pickup_country_name"),
                      col("country_code").alias("pickup_country_code"),
                      col("region").alias("pickup_region"),
                      col("region_code").alias("pickup_region_code"), "location_id")
                #################################################################
                #            
                #################################################################
                # this dataframe contains drop cordinates information
                cordinates_dropoff = coordinate_df.select(col("coordinate_id").alias("drop_coordinate_id"),
                       col("latitude").alias("drop_lattitude"),
                       col("longitude").alias("drop_longitude"),
                       col("country_name").alias("drop_country_name"),
                       col("country_code").alias("drop_country_code"),
                       col("region").alias("drop_region"),
                       col("region_code").alias("drop_region_code"), "location_id")
                #################################################################
                #            
                #################################################################
                yellow_with_Only_pickup_cordinates = yellow_taxi_trip_df.join(cordinates_pickup,
                    cordinates_pickup.location_id == yellow_taxi_trip_df.pu_location_id).select(
                        "trip_id", "pickup_lattitude",
                        "pickup_longitude",
                        "pickup_country_name",
                        "pickup_country_code",
                        "pickup_region",
                        "pickup_region_code")
                #################################################################
                #            
                #################################################################
                yellow_with_Only_dropoff_cordinates = yellow_taxi_trip_df.join(cordinates_dropoff,
                    cordinates_dropoff.location_id == yellow_taxi_trip_df.do_location_id).select(
                    "trip_id", "drop_lattitude",
                    "drop_longitude",
                    "drop_country_name",
                    "drop_country_code",
                    "drop_region",
                    "drop_region_code")
                
                #################################################################
                #            
                #################################################################
                processed_df = yellow_with_Only_pickup_cordinates.join(yellow_with_Only_dropoff_cordinates, ["trip_id"]).join(combined_df, ["trip_id"])
                #################################################################
                #            
                #################################################################
                vendor_map={1:"Creative Mobile Technologies, LLC",2:"VeriFone Inc."}
                
                
                #################################################################
                #            
                #################################################################
                # adding vendor name associated with corresponding vendor_id and adding column for the vendor name
                final_processed_df = processed_df.withColumn("vendor_name",when(processed_df.vendor_id==1,vendor_map.get(1)).when(processed_df.vendor_id==2,vendor_map.get(2)).otherwise("no such vendor"))
                
                #################################################################
                #            
                #################################################################
                final_processed_df.select("vendor_name","vendor_id","trip_id").show()
                
                
                
                #################################################################
                #            
                #################################################################
                print(final_processed_df)
                
                #################################################################
                #            
                #################################################################
                # changing the sequence of the columns to make it more readable
                final_processed_df=final_processed_df.select(col("trip_id"),col("vendor_id"),col("vendor_name"),col("pu_location_id"),col("pickup_zone"),
                          col("pickup_service_zone"),col("pickup_region_code"),col("pickup_region"),col("pickup_country_code"),
                          col("pickup_country_name"),col("pickup_lattitude"),col("pickup_longitude"),col("pickup_borough"),
                          col("tpep_pickup_datetime"),col("do_location_id"),col("drop_region_code"),col("drop_region"),col("drop_country_code"),
                          col("drop_country_name"),col("drop_zone"),col("drop_service_zone"),col("drop_lattitude"),col("drop_longitude"),
                          col("drop_borough"),col("tpep_dropoff_datetime"),col("rate_code_id"),col("type"),col("passenger_count"),col("trip_distance"),col("store_and_fwd_flag"),
                          col("payment_type_id"),col("mode"),col("congestion_surcharge"),col("improvement_surcharge"),col("tolls_amount"),
                          col("tip_amount"),col("mta_tax"),col("fare_amount"),col("extra"),col("total_amount")
                         )
                #################################################################
                #            
                #################################################################
                print(final_processed_df.count())
                final_processed_df.select('tpep_pickup_datetime','tpep_dropoff_datetime').show()
                
                #################################################################
                #            
                #################################################################
                final_processed_df.select("vendor_id","vendor_name","trip_id").show()
                
                #################################################################
                #            
                #################################################################
                len(final_processed_df.columns)
                
                #################################################################
                #            
                #################################################################
                # writing single processed parquet file
                final_processed_df.repartition(1).write.format("parquet").mode("append").save(f"/opt/notebooks/nyc/processed_zone/trips/year={year}/month={month}/day={day}/first.parquet")

                
                
                
print(trip_val)
final_processed_df.repartition(1).write.format("parquet").mode("append").save(f"/opt/notebooks/nyc/processed_zone/trips/year={year}/month={month}/day={day}/first.parquet")
                                                           

2022
Apr
+--------------------+---------+-------+
|         vendor_name|vendor_id|trip_id|
+--------------------+---------+-------+
|       VeriFone Inc.|      2.0|  299.0|
|Creative Mobile T...|      1.0|  305.0|
|       VeriFone Inc.|      2.0|  496.0|
|Creative Mobile T...|      1.0|  558.0|
|Creative Mobile T...|      1.0|  596.0|
|       VeriFone Inc.|      2.0|  692.0|
|       VeriFone Inc.|      2.0|  769.0|
|Creative Mobile T...|      1.0|  934.0|
|       VeriFone Inc.|      2.0| 1051.0|
|Creative Mobile T...|      1.0| 1761.0|
|       VeriFone Inc.|      2.0| 2734.0|
|       VeriFone Inc.|      2.0| 2815.0|
|       VeriFone Inc.|      2.0| 2862.0|
|       VeriFone Inc.|      2.0| 3597.0|
|Creative Mobile T...|      1.0| 3901.0|
|       VeriFone Inc.|      2.0| 3980.0|
|       VeriFone Inc.|      2.0| 4066.0|
|Creative Mobile T...|      1.0| 4142.0|
|       VeriFone Inc.|      2.0| 4800.0|
|       VeriFone Inc.|      2.0| 5360.0|
+--------------------+---------+-------+
only sh

# joining the tables

In [15]:
# final_processed_df.select("vendor_name","vendor_id","trip_id").show()

In [16]:
# final_processed_df.write.mode("append").parquet(f"/opt/notebooks/nyc/processed_zone/trips/year={year}/month={month}/day={day}/first.parquet")